In [1]:
# replace the standard sqlite3 module with pysqlite3
# for compatibility with Chroma
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import json
import langchain
import os
import bs4
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain_core.messages import SystemMessage
from langgraph.checkpoint import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

# load environment variables
load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# credential json not required if you are working within vertex AI workbench
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/LLM-agent-with-Gemini/fleet-anagram-244304-7dafcc771b2f.json"

LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # only if you are using text embedding model from google

In [3]:
llm = ChatVertexAI(model="gemini-1.5-pro")

In [ ]:

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are a helpful assistant that can help me to complete the following API payload:"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here are some examples of interactions you might have with me:

Q: B16C, 1126911, FPP
A: name:p.dsid,value, value:B16C, name:p.lot, value:1126911, name:p.type, value:FPP

Q: Y42M, 11952591, FQQP
A: name:p.dsid, value:Y42M, name:p.lot, value:11952591, name:p.type, value:FQQP

Q: Y42M, 1252391, FPC
A: name:p.dsid, value:Y42M, name:p.lot, value:1252391, name:p.type, value:FPC

Wrap the answer in a dictionary with the structure of the example above. The input will be a string with the format "dsid, lot, type" 
and the output should be a dictionary with the keys "name:p.dsid", "name:p.lot", and "name:p.type" with the corresponding values from the input string.

"""


example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

In [ ]:
print(
    pipeline_prompt.format(
        input="B16C, 1126911, FPP",
    )
)

In [ ]:
chain = pipeline_prompt | llm | JsonOutputParser()

output = chain.invoke({"input": "B47R,1952591,FPC"})
print(output)


In [ ]:
def format_input(input_string):
    parts = input_string.replace('.', '').split(',')
    parts = [part.strip() for part in parts if part.strip() != '']
    if len(parts) != 3:
        raise ValueError("Please check that the input contains exactly three parts: dsid, lot, and pid.")
    for part in parts:
        if part[0].isalpha() and part[-1].isalpha() and part[1:-1].isdigit():
            dsid = part
        elif part.isdigit():
            lot = part
        else:
            type_ = part
    output = {
        "name:p.dsid": dsid,
        "name:p.lot": lot,
        "name:p.type": type_
    }
    return output

In [ ]:
format_input("1952591,,,,,B47R,FPC")

In [ ]:
look_up_dictionary = {
    "Map viewer for detecting defects":"https://www.python.org/",
    "time series trends":"http://python.langchain.com"
}

In [13]:
class GenericInputSchema(BaseModel):
    """Inputs for url navigation tool."""
    domain: str = Field(
        description="The domain of the website you want to navigate to.")

class ParamInputSchema(BaseModel):
    """Inputs API payload."""
    params: str = Field(
        description="The input string with the format 'dsid, lot, type', which can be used to create a dictionary with the keys 'name:p.dsid', 'name:p.lot', and 'name:p.type'.")
    
class payload_formatter(BaseTool):
    name: str = "payload_formatter"
    args_schema: Optional[Type[BaseModel]] = ParamInputSchema
    description: str = """
    
    payload formatter is a tool that takes an input string with the format 'dsid, lot, type' and returns a dictionary with the keys 'name:p.dsid', 'name:p.lot', and 'name:p.type'.
    """
    def format_input(self,input_string):
        parts = input_string.replace('.', '').split(',')
        parts = [part.strip() for part in parts if part.strip() != '']
        if len(parts) != 3:
            raise ValueError("Please check that the input contains exactly three parts: dsid, lot, and pid.")
        for part in parts:
            if part[0].isalpha() and part[-1].isalpha() and part[1:-1].isdigit():
                dsid = part
            elif part.isdigit():
                lot = part
            else:
                type_ = part
        output = {
            "name:p.dsid": dsid,
            "name:p.lot": lot,
            "name:p.type": type_
        }
        return output

    def _run(self, params: str):
        return self.format_input(params)
    
class url_navigator(BaseTool):
    name: str = "map_viewer_url"
    args_schema: Optional[Type[BaseModel]] = GenericInputSchema
    description: str = """
    
    url navigator is a tool that takes a domain as input and returns the url of the website you want to navigate to.
    """

    def _run(self, domain: str):
        return f"www.{domain}.com"

In [14]:
tools = [url_navigator(), payload_formatter()]

In [15]:
system_message = "You are a helpful assistant and you know how to use url navigator tool and payload formatter tool."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

memory = MemorySaver()
app = create_react_agent(
    llm, tools, messages_modifier=system_message, checkpointer=memory
)


In [17]:
config = {"configurable": {"thread_id": "test-thread"}}

print(
    app.invoke(
        {
            "messages": [
                ("user", "1952591,,,,,B47R,FPC")
            ]
        },
        config,
    )["messages"][-1].content
)


print(
    app.invoke(
        {
            "messages": [
                ("Navigate to langchain")
            ]
        },
        config,
    )["messages"][-1].content
)

{'name:p.dsid': 'B47R', 'name:p.lot': '1952591', 'name:p.type': 'FPC'}
www.langchain.com
